<a href="https://colab.research.google.com/github/ChintPatel/CMPE258-HW7/blob/main/langgraph_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet langchain_core langchain-anthropic langgraph langsmith



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 11.5 MB/s eta 0:00:00


In [5]:
!pip install --quiet transformers accelerate torch langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00


In [28]:
!pip install --quiet google-genai langgraph
from google import genai
from google.genai import types
import os
from getpass import getpass
from langgraph.graph import StateGraph, START, END

if not os.environ.get("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("GEMINI_API_KEY: ")

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])


GEMINI_API_KEY: ··········


▶️ Cell 2: Configure LLM + Tracer

In [29]:
# Colab ▶️ Cell 2: Define a Gemini wrapper using the Google Gen AI SDK

def call_gemini(prompt: str) -> str:
    """
    Sends `prompt` to Gemini via the Google Gen AI SDK
    (free-tier API key) and returns the generated text.
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash",    # or gemini-1.5-flash, gemini-2.0-vision, etc.
        contents=prompt,             # can also pass a list for multimodal: [prompt, image_bytes]
        # Optionally control length/temperature:
        # config=types.GenerateContentConfig(
        #     max_output_tokens=128,
        #     temperature=0.7
        # )
    )
    return response.text

# Quick sanity check:
print(call_gemini("Write a short joke about cats"))


Why don't cats play poker in the jungle? 

Too many cheetahs! 



▶️ Cell 3: Prompt‐Chaining Workflow

In [33]:
# Colab ▶️ Cell 3: Prompt-Chaining Agent

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

class JokeState(TypedDict):
    topic: str
    joke: str
    improved_joke: str

def gen_joke(state: JokeState):
    return {"joke": call_gemini(f"Write a short joke about {state['topic']}")}

def has_punchline(state: JokeState) -> str:
    return "Pass" if any(p in state["joke"] for p in ("?", "!")) else "Fail"

def improve_joke(state: JokeState):
    return {"improved_joke": call_gemini(f"Make this joke funnier: {state['joke']}")}

graph1 = StateGraph(JokeState)
graph1.add_node("gen_joke", gen_joke)
graph1.add_conditional_edges("gen_joke", has_punchline, {
    "Pass": END,
    "Fail": "improve_joke"
})
graph1.add_node("improve_joke", improve_joke)
graph1.add_edge(START, "gen_joke")
graph1.add_edge("improve_joke", END)

chain1 = graph1.compile()


▶️ Cell 4: Invoke Prompt‐Chaining & View Trace

In [34]:
# Colab ▶️ Cell 4: Run Prompt-Chaining

result1 = chain1.invoke({"topic": "cats"})
print("Final joke:", result1.get("joke") or result1.get("improved_joke"))


Final joke: Why did the cat join the Red Cross? 

Because he wanted to be a first-aid kit! 



▶️ Cell 5: Parallelization Workflow

In [35]:
# Colab ▶️ Cell 5: Parallelization Agent

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

class ParState(TypedDict):
    topic: str
    joke: str
    story: str
    poem: str
    combined: str  # include for the final step

def call_joke(state: ParState):
    return {"joke": call_gemini(f"Write a joke about {state['topic']}")}

def call_story(state: ParState):
    return {"story": call_gemini(f"Write a short story about {state['topic']}")}

def call_poem(state: ParState):
    return {"poem": call_gemini(f"Write a poem about {state['topic']}")}

def combine_all(state: ParState):
    return {
        "combined": (
            f"JOKE:\n{state['joke']}\n\n"
            f"STORY:\n{state['story']}\n\n"
            f"POEM:\n{state['poem']}"
        )
    }

graph2 = StateGraph(ParState)
# add the three LLM calls with unique node names
graph2.add_node("call_joke", call_joke)
graph2.add_node("call_story", call_story)
graph2.add_node("call_poem", call_poem)
graph2.add_node("combine_all", combine_all)

# fan-in structure
for node in ("call_joke", "call_story", "call_poem"):
    graph2.add_edge(START, node)
    graph2.add_edge(node, "combine_all")
graph2.add_edge("combine_all", END)

chain2 = graph2.compile()


In [36]:
# Colab ▶️ Cell 6: Run Parallelization

result2 = chain2.invoke({"topic": "cats"})
print(result2["combined"])


JOKE:
Why did the cat join the Red Cross?

Because he wanted to learn first aid!
...You know, for when he gets into cat fights.


STORY:
The alley was their kingdom, a labyrinth of overflowing bins, sun-baked brick, and the comforting smell of forgotten things. Whiskers, a grizzled tabby with one ear permanently bent, ruled with the quiet authority of experience. He'd seen seasons come and go, kittens born and lost, and knew every crevice where a scrap of food could be found or a moment of peace stolen.

Tonight, though, a tremor of unrest ran through the feline court. A newcomer had arrived. Midnight, a sleek, black she-cat with emerald eyes that pierced the darkness, had been scavenging at the edge of their territory for days. Whiskers had tolerated her presence, understanding the primal need for survival, but tonight, she was bolder. She sat right at the mouth of the alley, her gaze fixed on the overflowing bin where Whiskers kept his most prized treasure – a discarded, half-eaten t